Run a quick simulation to view the A matrix

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from getParam_Sonar import getParam_Sonar
from matplotlib.patches import Rectangle

Nx, Nz = 20, 10 
Lx, Lz = 200, 100 

p, x_start, t_start, t_stop, max_dt_FE = getParam_Sonar(Nx, Nz, Lx, Lz, UseSparseMatrices=False)

A = p['A']
B = p['B']
N = Nx * Nz

print(f"System size: {A.shape}")
print(f"Grid: {Nx} x {Nz} = {N} nodes")
print(f"State vector size: {2*N}")
print(f"Source at grid point: ({p['sonar_ix']}, {p['sonar_iz']})")

# Visualize sparsity pattern
fig, ax = plt.subplots(figsize=(6, 6))

ax.spy(A, markersize=0.5)
ax.set_title(f'Sparsity Pattern (density: {np.count_nonzero(A)/A.size:.2%})')
ax.set_xlabel('State index')
ax.set_ylabel('State index')

# Outline the L block (bottom-left N x N)
ax.add_patch(Rectangle((0, N), N, N, fill=False, edgecolor='red', linewidth=1.8))

# Label it
ax.text(N*.9, N*1.1, r'$L$', color='red', ha='center', va='center', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

# 2-norm condition number: κ₂(A) = σ_max / σ_min
kappa2_A = np.linalg.cond(A)             
print(f"cond₂(A) = {kappa2_A:.3e}")
print(f"A dtype: {A.dtype}")
print(f"Max decimal digits for {A.dtype}: {np.finfo(A.dtype).precision} so can work with at most ~{np.finfo(A.dtype).precision - np.log10(kappa2_A):.0f} digits of precision")


# What you see:
#   d[p]/dt = [0   I ][p] + [0]u
#    [v]      [L  -aI][v]   [b]

Next, run provided SimpleSolver. We don't expect stable behavior, since Forward Euler is non-ideal for wave propagation.

In [ ]:
from eval_f_Sonar import eval_f_Sonar
from eval_u_Sonar import eval_u_Sonar_20, eval_u_Sonar_20_const
from SimpleSolver import SimpleSolver

Nx, Nz = 100, 50    # Nx, Nz = 20, 20  
Lx, Lz = 10e3, 1e3   # Lx, Lz = 50, 50  

p, x_start, t_start, t_stop, max_dt_FE = getParam_Sonar(Nx, Nz, Lx, Lz, UseSparseMatrices=True)

eval_u = eval_u_Sonar_20 # eval_u_Sonar_20_const
# Scale u(t) by cell area so Bu remains grid-invariant with b=1/(dx*dz)
def eval_u_scaled(t):
    return (p['dx']*p['dz']) * eval_u(t)
eval_f = eval_f_Sonar

visualize = False
w = max_dt_FE * 0.1
num_iter = int(np.ceil((t_stop - t_start) / w)) 

print(f"Number of iterations: {num_iter}")

[X, t] = SimpleSolver(eval_f, x_start, p, eval_u, num_iter, w, visualize, gif_file_name="test_SimpleSolver_Sonar.gif")

In [ ]:
from visualize_sonar import visualize_sonar_setup

visualize_sonar_setup(p, show_grid=True, show_boundary_labels=True)

# hydrophone

In [ ]:
from eval_g_Sonar import eval_g_Sonar

# extract hydrophone outputs over time
print("Processing hydrophone data...")
num_phones = len(p['hydrophones']['x_indices'])
hydrophone_history = np.zeros((num_phones, len(t)))

for i in range(len(t)):
    y = eval_g_Sonar(X[:, i], p)
    hydrophone_history[:, i] = y.flatten()


fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

adjust_view = 1e7 #1e9
# Auto-scale waveform display so traces are visible relative to offsets
amp = float(np.nanmax(np.abs(hydrophone_history)) + 1e-12)
adjust_view = 50.0 / amp
# Auto-scale waveform display so traces are visible relative to offsets
amp = float(np.nanmax(np.abs(hydrophone_history)) + 1e-12)
adjust_view = 50.0 / amp

# waterfall plot 
for i in range(num_phones):
    x_pos = p['hydrophones']['x_indices'][i] * p['dx']
    ax1.plot(t*1000, adjust_view*hydrophone_history[i, :] + i*100, 'b-', alpha=0.7)
    
ax1.set_xlabel('Time (ms)')
ax1.set_ylabel('Hydrophone Index + Offset')
ax1.set_title('Hydrophone Array Response (Waterfall View)')
ax1.grid(True, alpha=0.3)

# heatmap 
vmax = float(np.quantile(np.abs(hydrophone_history), 0.995) + 1e-12)
vmin = -vmax
im = ax2.imshow(hydrophone_history, aspect='auto', origin='lower',
                extent=[t[0]*1000, t[-1]*1000, 0, num_phones-1],
                cmap='RdBu_r')
im.set_clim(vmin=vmin, vmax=vmax)
ax2.set_xlabel('Time (ms)')
ax2.set_ylabel('Hydrophone Index')
ax2.set_title('Hydrophone Array Response (Heatmap)')
plt.colorbar(im, ax=ax2, label='Pressure (Pa)')

plt.tight_layout()
plt.show()

In [ ]:
from simpleLeapFrog import *

X, t = LeapfrogSolver(eval_f, x_start, p, eval_u_scaled, num_iter, dt=w, visualize=False)

In [ ]:
from eval_g_Sonar import eval_g_Sonar

# extract hydrophone outputs over time
print("Processing hydrophone data...")
num_phones = len(p['hydrophones']['x_indices'])
hydrophone_history = np.zeros((num_phones, len(t)))

for i in range(len(t)):
    y = eval_g_Sonar(X[:, i], p)
    hydrophone_history[:, i] = y.flatten()


fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

adjust_view = 1e4 #1e9

# waterfall plot 
for i in range(num_phones):
    x_pos = p['hydrophones']['x_indices'][i] * p['dx']
    ax1.plot(t*1000, adjust_view*hydrophone_history[i, :] + i*100, 'b-', alpha=0.7)
    
ax1.set_xlabel('Time (ms)')
ax1.set_ylabel('Hydrophone Index + Offset')
ax1.set_title('Hydrophone Array Response (Waterfall View)')
ax1.grid(True, alpha=0.3)

# heatmap 
vmax = float(np.quantile(np.abs(hydrophone_history), 0.995) + 1e-12)
vmin = -vmax
im = ax2.imshow(hydrophone_history, aspect='auto', origin='lower',
                extent=[t[0]*1000, t[-1]*1000, 0, num_phones-1],
                cmap='RdBu_r')
im.set_clim(vmin=vmin, vmax=vmax)
ax2.set_xlabel('Time (ms)')
ax2.set_ylabel('Hydrophone Index')
ax2.set_title('Hydrophone Array Response (Heatmap)')
plt.colorbar(im, ax=ax2, label='Pressure (Pa)')

plt.tight_layout()
plt.show()

In [ ]:
# Compare solvers
fig = test_solver_stability(p, eval_f, eval_u, x_start, max_dt_FE)
plt.show()

Implement our own suite of tests

In [ ]:
from test_Sonar import test_sonar_complete

test_sonar_complete(eval_u)

# finite differences Jacobian is sitting right around machine precision
# our system is indeed linear! no truncation error

In [ ]:
# Test dt sweep for leapfrog
fig = test_dt_sweep_leapfrog(eval_f_Sonar, x_start, p, eval_u,
                             exponents=range(-9, -13, -1),
                             max_steps=100,
                             use_outputs=True,
                             quiet=False)
plt.show()

# wave viz

In [ ]:
# Visualize pressure over XZ plane at a given time/index
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
from sonar_viz import plot_pressure_xz_at

# Interactive index slider if ipywidgets is available; fallback to last frame
try:
    import ipywidgets as widgets
    slider = widgets.IntSlider(value=int(len(np.asarray(t).reshape(-1)) - 1), min=0, max=int(X.shape[1] - 1), step=1, description='Index')
    ui = widgets.HBox([slider])
    out = widgets.interactive_output(lambda Index: plot_pressure_xz_at(p, X, t, index=Index), {'Index': slider})
    display(ui, out)
except Exception as e:
    print('Interactive widgets unavailable; plotting last step. Error:', e)
    plot_pressure_xz_at(p, X, t, index=int(X.shape[1] - 1))

In [ ]:
import create_wave_animation as cwa
cwa.create_wave_animation(X, t, p, save_filename='wave_animation.gif')

In [ ]:
import create_3d_wave_animation as cwa3d
cwa3d.create_3d_wave_animation(X, t, p, save_filename='wave_animation_3d.gif')